# 3.6 손글씨 숫자 인식
## 3.6.1 MNIST 데이터셋
- 손글씨 숫자 이미지 집합
- 0-9 숫자 이미지로 구성
- 훈련 이미지(for 모델 학습) 60000장, 시험 이미지(for 학습한 모델로 시험 이미지를 얼마나 정확하게 분류하는가 판별) 10000장

In [1]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉토리 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [2]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


- `load_mnist()`: MNIST 데이터를 (훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블) 형식으로 변환
    - normalize: 입력 이미지의 픽셀 값을 [0.0, 1.0] 값으로 정규화할 것인가
    - flatten: 입력 이미지를 1차원 배열로 만들 것인지
    - one_hot_label: label을 one-hot encoding 형태로 저장할지

In [3]:
import numpy as np
from PIL import Image  # PIL: Python Image Library

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)  # 원래 이미지의 모양으로 변형
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


- `flatten=True`이므로 이미지는 1차원 넘파이 배열로 저장 -> 이미지 표시할 때 28 * 28 크기로 다시 변형해야 함. -> `reshape()`로 넘파이 배열의 형상을 바꿀 수 있음
- `Image.fromarray()`: 넘파이로 저장된 이미지 데이터를 PIL용 데이터 객체로 변환

### 3.6.2 신경망의 추론 처리
- 입력층 뉴런 784(=28 * 28)개, 출력층 뉴런 10개, 은닉층 2개(1번째 은닉층 뉴런 50개, 2번째 은닉층 뉴런 100개)인 신경망

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a)
    
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

In [5]:
import pickle

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("dataset/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

- `init_network()`: sample_weight.pkl(가중치와 편향 매개변수가 딕셔너리로 저장)에 저장된 학습된 가중치 매개변수 읽어들이기

In [6]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)  # 확률이 가장 높은 원소의 인덱스
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy: " + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


- `predict()`: 각 레이블의 확률을 넘파이 배열로 반환
- `np.argmax()`: 배열에서 최대값(가장 높은 확률)의 원소의 인덱스 반환 -> 예측 결과
- `accuracy_cnt`: 신경망이 예측한 답변과 정답 레이블 비교해 맞힌 횟수 세기 -> 전체 이미지 숫자로 나눠 정확도 구하기

- 정규화(normalization): 데이터를 특정 범위로 변환하는 처리
- 전처리(pre-processing): 신경망의 입력 데이터에 특정 변환을 가하는 것

### 3.6.3 배치(Batch) 처리

In [7]:
# 구현한 신경망 각 층의 가중치 형상 출력
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


- 다차원 배열에서 대응하는 차원의 원소 수가 일치함

![신경망 각 층의 배열 형상 추이](https://user-images.githubusercontent.com/61455647/114272320-b0405780-9a50-11eb-9ad5-070957962491.png)

- 데이터 1장만 입력했을 때, 입력: 원소 784(=28 * 28)개로 구성된 1차원 배열 | 출력: 원소 10개인 1차원 배열

![배치 처리를 위한 배열들의 형상 추이](https://user-images.githubusercontent.com/61455647/114272469-47a5aa80-9a51-11eb-8ca1-839b5cc6f544.png)

- 이미지 100개일 때, 입력: 100 * 784개 | 출력: 100 * 10개
- **배치 Batch**: 하나로 묶은 입력 데이터
- 배치 처리의 장점: 이미지 1장당 처리 시간 ↓
    - 이유1. 수치 계산 라이브러리는 큰 배열을 효율적으로 처리할 수 있도록 최적화 되어 있음
    - 이유2. I/O로 데이터를 읽는 횟수가 줄어 배치 처리로 버스에 부하를 줄일 수 있음

In [8]:
# 배치 처리 구현
x, t = get_data()
network = init_network()

batch_size = 100  # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy: " + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


- `range(start, end, step)`: start~end-1까지 step 간격으로 증가하는 리스트 반환
- `x[i:i+batch_size]`: 입력 데이터의 i번째~i+batch_size번째의 데이터 묶기
- `argmax(, axis=1)`: 100 * 10 배열 중 1번째 차원을 구성하는 각 원소에서 최대값의 인덱스
- `accuracy_cnt += np.sum(p == t[i:i+batch_size])`: 배치 단위로 분류한 결과와 실제 답을 비교 -> bool 배열에서 True 개수 세서 반환

In [9]:
# argmax(, axis=) 예시
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=0)
print(y)
y = np.argmax(x, axis=1)
print(y)

[3 0 1]
[1 2 1 0]


- 데이터를 배치로 처리해 효율적이고 빠르게 처리할 수 있음